# Managing pods’ computational resources

Up to now you’ve created pods without caring about how much CPU and memory they’re allowed to consume. But as you’ll see in this chapter, setting both how much a pod is expected to consume and the maximum amount it’s allowed to consume is a vital part of any pod definition. Setting these two sets of parameters makes sure that a pod takes only its fair share of the resources provided by the Kubernetes cluster and also affects how pods are scheduled across the cluster.

## Requesting resources for a pod’s containers

When creating a pod, you can specify the amount of CPU and memory that a container needs (these are called requests) and a hard limit on what it may consume (known as limits). They’re specified for each container individually, not for the pod as a whole. The pod’s resource requests and limits are the sum of the requests and limits of all its containers.

### Creating pods with resource requests

Let’s look at an example pod manifest, which has the CPU and memory requests specified for its single container, as shown in the following listing.

```yml
apiVersion: v1
kind: Pod
metadata:
  name: requests-pod
spec:
  containers:
  - image: busybox
    command: ["dd", "if=/dev/zero", "of=/dev/null"]
    name: main
    resources:
      requests:
        cpu: 200m
        memory: 10Mi
```

    kubectl apply -f ex01-requests-pod.yaml

In the pod manifest, your single container requires one-fifth of a CPU core (200 millicores) to run properly. Five such pods/containers can run sufficiently fast on a single CPU core.

When you don’t specify a request for CPU, you’re saying you don’t care how much CPU time the process running in your container is allotted. **In the worst case, it may not get any CPU time at all** (this happens when a heavy demand by other processes exists on the CPU). Although this may be fine for low-priority batch jobs, which aren’t time-critical, it obviously isn’t appropriate for containers handling user requests.

In the pod spec, you’re also requesting 10 mebibytes of memory for the container. By doing that, you’re saying that you expect the processes running inside the container to use at most 10 mebibytes of RAM. They might use less, but you’re not expecting them to use more than that in normal circumstances. Later in this chapter you’ll see what happens if they do.

Now you’ll run the pod. When the pod starts, you can take a quick look at the process’ CPU consumption by running the top command inside the container, as shown in the following listing.

    kubectl exec -it requests-pod -- top

> Vsak node ima 1 CPU

    Mem: 3614728K used, 171212K free, 1428K shrd, 146328K buff, 2565716K cached
    CPU: 47.8% usr 52.2% sys  0.0% nic  0.0% idle  0.0% io  0.0% irq  0.0% sirq
    Load average: 1.18 0.33 0.15 2/441 10
      PID  PPID USER     STAT   VSZ %VSZ CPU %CPU COMMAND
        1     0 root     R     1300  0.0   0 96.4 dd if /dev/zero of /dev/null
        6     0 root     R     1308  0.0   0  0.0 top

The dd command you’re running in the container consumes as much CPU as it can, but it only runs a single thread so it can only use a single core. The Minikube VM, which is where this example is running, has one CPU core allotted to it. That’s why the process is shown consuming 100% of the whole CPU.

Fifty percent of two cores is obviously one whole core, which means the container is using more than the 200 millicores you requested in the pod specification. This is expected, because requests don’t limit the amount of CPU a container can use. You’d need to specify a CPU limit to do that. You’ll try that later, but first, let’s see how specifying resource requests in a pod affects the scheduling of the pod.

### Understanding how resource requests affect scheduling

By `specifying resource requests, you’re specifying the minimum amount of resources your pod needs`. This information is what the Scheduler uses when scheduling the pod to a node. Each node has a certain amount of CPU and memory it can allocate to pods. When scheduling a pod, the Scheduler will only consider nodes with enough unallocated resources to meet the pod’s resource requirements. If the amount of unallocated CPU or memory is less than what the pod requests, Kubernetes will not schedule the pod to that node, because the node can’t provide the minimum amount required by the pod.

**Understanding how the Scheduler determines if a pod can fit on a node**

What’s important and somewhat surprising here is that the Scheduler `doesn’t look at how much of each individual resource is being used at the exact time of scheduling but at the sum of resources requested by the existing pods deployed on the node`. Even though existing pods may be using less than what they’ve requested, scheduling another pod based on actual resource consumption would break the guarantee given to the already deployed pods.

This is visualized in figure 14.1. Three pods are deployed on the node. Together, they’ve requested 80% of the node’s CPU and 60% of the node’s memory. Pod D, shown at the bottom right of the figure, cannot be scheduled onto the node because it requests 25% of the CPU, which is more than the 20% of unallocated CPU. The fact that the three pods are currently using only 70% of the CPU makes no difference.

This is visualized in figure 14.1. Three pods are deployed on the node. Together, they’ve requested 80% of the node’s CPU and 60% of the node’s memory. Pod D, shown at the bottom right of the figure, cannot be scheduled onto the node because it requests 25% of the CPU, which is more than the 20% of unallocated CPU. The fact that the three pods are currently using only 70% of the CPU makes no difference.

<img alt="" src="https://dpzbhybb2pdcj.cloudfront.net/luksa/Figures/14fig01_alt.jpg" data-action="zoom" data-zoom-src="https://dpzbhybb2pdcj.cloudfront.net/luksa/HighResolutionFigures/figure_14-1.png" class="medium-zoom-image">

**Understanding how the Scheduler uses pods’ requests when selecting the best node for a pod**

You may remember from chapter 11 that the Scheduler first filters the list of nodes to exclude those that the pod can’t fit on and then prioritizes the remaining nodes per the configured prioritization functions. Among others, two prioritization functions rank nodes based on the amount of resources requested: LeastRequestedPriority and MostRequestedPriority. The first one prefers nodes with fewer requested resources (with a greater amount of unallocated resources), whereas the second one is the exact opposite—it prefers nodes that have the most requested resources (a smaller amount of unallocated CPU and memory). But, as we’ve discussed, they both consider the amount of requested resources, not the amount of resources actually consumed.

The Scheduler is configured to use only one of those functions. You may wonder why anyone would want to use the MostRequestedPriority function. After all, if you have a set of nodes, you usually want to spread CPU load evenly across them. However, that’s not the case when running on cloud infrastructure, where you can add and remove nodes whenever necessary. By configuring the Scheduler to use the Most-RequestedPriority function, you guarantee that Kubernetes will use the smallest possible number of nodes while still providing each pod with the amount of CPU/memory it requests. By keeping pods tightly packed, certain nodes are left vacant and can be removed. Because you’re paying for individual nodes, this saves you money.

**Inspecting a node’s capacity**

Let’s see the Scheduler in action. You’ll deploy another pod with four times the amount of requested resources as before. But before you do that, let’s see your node’s capacity. Because the Scheduler needs to know how much CPU and memory each node has, the Kubelet reports this data to the API server, making it available through the Node resource. You can see it by using the kubectl describe command as in the following listing.

    kubectl describe nodes

The output shows two sets of amounts related to the available resources on the node: the node’s capacity and allocatable resources. The capacity represents the total resources of a node, which may not all be available to pods. Certain resources may be reserved for Kubernetes and/or system components. The Scheduler bases its decisions only on the allocatable resource amounts.

In the previous example, the node called minikube runs in a VM with two cores and has no CPU reserved, making the whole CPU allocatable to pods. Therefore, the Scheduler should have no problem scheduling another pod requesting 800 millicores.

Run the pod now. You can use the YAML file in the code archive, or run the pod with the kubectl run command like this:

    kubectl run requests-pod-2 --image=busybox --restart Never --requests='cpu=500m,memory=20Mi' -- dd if=/dev/zero of=/dev/null

Let’s see if it was scheduled:

    kubectl get pod

Okay, the pod has been scheduled and is running.

**Creating a pod that doesn’t fit on any node**

You now have two pods deployed, which together have requested a total of 1,000 millicores or exactly 1 core. You should therefore have another 1,000 millicores available for additional pods, right? You can deploy another pod with a resource request of 1,000 millicores. Use a similar command as before:

    kubectl run requests-pod-3 --image=busybox --restart Never --requests='cpu=1,memory=20Mi' -- dd if=/dev/zero of=/dev/null

> NOTE: This time you’re specifying the CPU request in whole cores (cpu=1) instead of millicores (cpu=1000m).

So far, so good. The pod has been accepted by the API server (you’ll remember from the previous chapter that the API server can reject pods if they’re invalid in any way). Now, check if the pod is running:

    kubectl get po requests-pod-3

Even if you wait a while, the pod is still stuck at Pending. You can see more information on why that’s the case by using the kubectl describe command, as shown in the following listing.

    kubectl describe po requests-pod-3

The output shows that the pod hasn’t been scheduled because it can’t fit on any node due to insufficient CPU on your single node. But why is that? The sum of the CPU requests of all three pods equals 2,000 millicores or exactly two cores, which is exactly what your node can provide. What’s wrong?

**Determining why a pod isn’t being scheduled**

You can figure out why the pod isn’t being scheduled by inspecting the node resource. Use the kubectl describe node command again and examine the output more closely in the following listing.

    kubectl describe node

You can find the culprit in the list of pods in the previous listing. Three pods in the kube-system namespace have explicitly requested CPU resources. Those pods plus your two pods leave only 725 millicores available for additional pods. Because your third pod requested 1,000 millicores, the Scheduler won’t schedule it to this node, as that would make the node overcommitted.

**Freeing resources to get the pod scheduled**

The pod will only be scheduled when an adequate amount of CPU is freed (when one of the first two pods is deleted, for example). If you delete your second pod, the Scheduler will be notified of the deletion (through the watch mechanism described in chapter 11) and will schedule your third pod as soon as the second pod terminates. This is shown in the following listing.

    kubectl delete po requests-pod-2

    kubectl get po

In all these examples, you’ve specified a request for memory, but it hasn’t played any role in the scheduling because your node has more than enough allocatable memory to accommodate all your pods’ requests. Both CPU and memory requests are treated the same way by the Scheduler, but in contrast to memory requests, a pod’s CPU requests also play a role elsewhere—while the pod is running. You’ll learn about this next.

### Understanding how CPU requests affect CPU time sharing

You now have two pods running in your cluster (you can disregard the system pods right now, because they’re mostly idle). One has requested 200 millicores and the other one five times as much. At the beginning of the chapter, we said Kubernetes distinguishes between resource requests and limits. You haven’t defined any limits yet, so the two pods are in no way limited when it comes to how much CPU they can each consume. If the process inside each pod consumes as much CPU time as it can, how much CPU time does each pod get?

The CPU requests don’t only affect scheduling—they also determine how the remaining (unused) CPU time is distributed between pods. Because your first pod requested 200 millicores of CPU and the other one 1,000 millicores, any unused CPU will be split among the two pods in a 1 to 5 ratio, as shown in figure 14.2. If both pods consume as much CPU as they can, the first pod will get one sixth or 16.7% of the CPU time and the other one the remaining five sixths or 83.3%.

<img alt="" src="https://dpzbhybb2pdcj.cloudfront.net/luksa/Figures/14fig02_alt.jpg" data-action="zoom" data-zoom-src="https://dpzbhybb2pdcj.cloudfront.net/luksa/HighResolutionFigures/figure_14-2.png" class="medium-zoom-image">

But if one container wants to use up as much CPU as it can, while the other one is sitting idle at a given moment, the first container will be allowed to use the whole CPU time (minus the small amount of time used by the second container, if any). After all, it makes sense to use all the available CPU if no one else is using it, right? As soon as the second container needs CPU time, it will get it and the first container will be throttled back.

## Limiting resources available to a container

Setting resource requests for containers in a pod ensures each container gets the minimum amount of resources it needs. Now let’s see the other side of the coin—the maximum amount the container will be allowed to consume.

### Setting a hard limit for the amount of resources a container can use

We’ve seen how containers are allowed to use up all the CPU if all the other processes are sitting idle. But you may want to prevent certain containers from using up more than a specific amount of CPU. And you’ll always want to limit the amount of memory a container can consume.

CPU is a compressible resource, which means the amount used by a container can be throttled without affecting the process running in the container in an adverse way. Memory is obviously different—it’s incompressible. Once a process is given a chunk of memory, that memory can’t be taken away from it until it’s released by the process itself. That’s why you need to limit the maximum amount of memory a container can be given.

Without limiting memory, a container (or a pod) running on a worker node may eat up all the available memory and affect all other pods on the node and any new pods scheduled to the node (remember that new pods are scheduled to the node based on the memory requests and not actual memory usage). A single malfunctioning or malicious pod can practically make the whole node unusable.

**Creating a pod with resource limits**

To prevent this from happening, Kubernetes allows you to specify resource limits for every container (along with, and virtually in the same way as, resource requests). The following listing shows an example pod manifest with resource limits.

```yml
apiVersion: v1
kind: Pod
metadata:
  name: limited-pod
spec:
  containers:
  - image: busybox
    command: ["dd", "if=/dev/zero", "of=/dev/null"]
    name: main
    resources:
      limits:
        cpu: 0.5
        memory: 20Mi
            
```

This pod’s container has resource limits configured for both CPU and memory. The process or processes running inside the container will not be allowed to consume more than 1 CPU core and 20 mebibytes of memory.

> NOTE: Because you haven’t specified any resource requests, they’ll be set to the same values as the resource limits.

    kubectl apply -f ex02-limited-pod.yaml

    kubectl exec -it limited-pod -- top

**Overcommitting limits**

Unlike resource requests, resource limits aren’t constrained by the node’s allocatable resource amounts. The sum of all limits of all the pods on a node is allowed to exceed 100% of the node’s capacity (figure 14.3). Restated, resource limits can be overcommitted. This has an important consequence—when 100% of the node’s resources are used up, certain containers will need to be killed.

<img alt="" src="https://dpzbhybb2pdcj.cloudfront.net/luksa/Figures/14fig03_alt.jpg" data-action="zoom" data-zoom-src="https://dpzbhybb2pdcj.cloudfront.net/luksa/HighResolutionFigures/figure_14-3.png" class="medium-zoom-image">

You’ll see how Kubernetes decides which containers to kill in section 14.3, but individual containers can be killed even if they try to use more than their resource limits specify. You’ll learn more about this next.

### Exceeding the limits

What happens when a process running in a container tries to use a greater amount of resources than it’s allowed to?

You’ve already learned that CPU is a compressible resource, and it’s only natural for a process to want to consume all of the CPU time when not waiting for an I/O operation. As you’ve learned, a process’ CPU usage is throttled, so when a CPU limit is set for a container, the process isn’t given more CPU time than the configured limit.

With memory, it’s different. When a process tries to allocate memory over its limit, the process is killed (it’s said the container is OOMKilled, where OOM stands for Out Of Memory). If the pod’s restart policy is set to Always or OnFailure, the process is restarted immediately, so you may not even notice it getting killed. But if it keeps going over the memory limit and getting killed, Kubernetes will begin restarting it with increasing delays between restarts. You’ll see a CrashLoopBackOff status in that case:

    $ kubectl get po
    NAME        READY     STATUS             RESTARTS   AGE
    memoryhog   0/1       CrashLoopBackOff   3          1m



The CrashLoopBackOff status doesn’t mean the Kubelet has given up. It means that after each crash, the Kubelet is increasing the time period before restarting the container. After the first crash, it restarts the container immediately and then, if it crashes again, waits for 10 seconds before restarting it again. On subsequent crashes, this delay is then increased exponentially to 20, 40, 80, and 160 seconds, and finally limited to 300 seconds. Once the interval hits the 300-second limit, the Kubelet keeps restarting the container indefinitely every five minutes until the pod either stops crashing or is deleted.

To examine why the container crashed, you can check the pod’s log and/or use the kubectl describe pod command, as shown in the following listing.

    $ kubectl describe pod
    Name:       memoryhog
    ...
    Containers:
      main:
        ...
        State:          Terminated
          Reason:       OOMKilled
          Exit Code:    137
          Started:      Tue, 27 Dec 2016 14:55:53 +0100
          Finished:     Tue, 27 Dec 2016 14:55:58 +0100
        Last State:     Terminated
          Reason:       OOMKilled
          Exit Code:    137
          Started:      Tue, 27 Dec 2016 14:55:37 +0100
          Finished:     Tue, 27 Dec 2016 14:55:50 +0100
        Ready:          False
    ...



The OOMKilled status tells you that the container was killed because it was out of memory. In the previous listing, the container went over its memory limit and was killed immediately.

It’s important not to set memory limits too low if you don’t want your container to be killed. But containers can get OOMKilled even if they aren’t over their limit. You’ll see why in section 14.3.2, but first, let’s discuss something that catches most users off-guard the first time they start specifying limits for their containers.

###  Understanding how apps in containers see limits

Now, run the top command in the container, the way you did at the beginning of the chapter. The command’s output is shown in the following listing.

    $ kubectl exec -it limited-pod top
    Mem: 1450980K used, 597504K free, 22012K shrd, 65876K buff, 857552K cached
    CPU: 10.0% usr 40.0% sys  0.0% nic 50.0% idle  0.0% io  0.0% irq  0.0% sirq
    Load average: 0.17 1.19 2.47 4/503 10
      PID  PPID USER     STAT   VSZ %VSZ CPU %CPU COMMAND
        1     0 root     R     1192  0.0   1 49.9 dd if /dev/zero of /dev/null
        5     0 root     R     1196  0.0   0  0.0 top



examine the output of the top command closely. Is there anything that strikes you as odd?

Look at the amount of used and free memory. Those numbers are nowhere near the 20 MiB you set as the limit for the container. Similarly, you set the CPU limit to one core and it seems like the main process is using only 50% of the available CPU time, even though the dd command, when used like you’re using it, usually uses all the CPU it has available. What’s going on?

**Understanding that containers always see the node’s memory, not the container’s**

The top command shows the memory amounts of the whole node the container is running on. Even though you set a limit on how much memory is available to a container, the container will not be aware of this limit.

This has an unfortunate effect on any application that looks up the amount of memory available on the system and uses that information to decide how much memory it wants to reserve.

The problem is visible when running Java apps, especially if you don’t specify the maximum heap size for the Java Virtual Machine with the -Xmx option. In that case, the JVM will set the maximum heap size based on the host’s total memory instead of the memory available to the container. When you run your containerized Java apps in a Kubernetes cluster on your laptop, the problem doesn’t manifest itself, because the difference between the memory limits you set for the pod and the total memory available on your laptop is not that great.

But when you deploy your pod onto a production system, where nodes have much more physical memory, the JVM may go over the container’s memory limit you configured and will be OOMKilled.

And if you think setting the -Xmx option properly solves the issue, you’re wrong, unfortunately. The -Xmx option only constrains the heap size, but does nothing about the JVM’s off-heap memory. Luckily, new versions of Java alleviate that problem by taking the configured container limits into account.

**Understanding that containers also see all the node’s CPU cores**

Exactly like with memory, containers will also see all the node’s CPUs, regardless of the CPU limits configured for the container. Setting a CPU limit to one core doesn’t magically only expose only one CPU core to the container. All the CPU limit does is constrain the amount of CPU time the container can use.

A container with a one-core CPU limit running on a 64-core CPU will get 1/64th of the overall CPU time. And even though its limit is set to one core, the container’s processes will not run on only one core. At different points in time, its code may be executed on different cores.

Nothing is wrong with this, right? While that’s generally the case, at least one scenario exists where this situation is catastrophic.

Certain applications look up the number of CPUs on the system to decide how many worker threads they should run. Again, such an app will run fine on a development laptop, but when deployed on a node with a much bigger number of cores, it’s going to spin up too many threads, all competing for the (possibly) limited CPU time. Also, each thread requires additional memory, causing the apps memory usage to skyrocket.

You may want to use the Downward API to pass the CPU limit to the container and use it instead of relying on the number of CPUs your app can see on the system. You can also tap into the cgroups system directly to get the configured CPU limit by reading the following files:
- /sys/fs/cgroup/cpu/cpu.cfs_quota_us
- /sys/fs/cgroup/cpu/cpu.cfs_period_us